In [ ]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from fixed_hedge_strategy import FixedHedgeStrategy
from datetime import datetime

In [ ]:
params = {
    'hedge_range': 0.06,
    'hedge_multiple': 0.5,
    'hedge_size': 10
}

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="510050.SSE",
    interval="1m",
    start=datetime(2018, 1, 1),
    end=datetime(2020, 4, 1),
    rate=0.1/10000,
    slippage=0.001,
    size=10000,
    pricetick=0.001,
    capital=1_000_000,
)
engine.add_strategy(FixedHedgeStrategy, params)

In [ ]:
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()

In [ ]:
import numpy as np

In [ ]:
np.random.randint(-6, 5) / 100

In [ ]:
round(2.739, 2)